In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import time
import datetime

In [ ]:
def abs_MAPE_error(y_pred,y_true):
    return np.mean((np.abs(y_pred-y_true)/np.abs(y_true)) )
def abs_MAPE_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true)/np.abs(y_true)
def abs_error(y_pred,y_true):
    return np.mean((np.abs(y_pred-y_true)))  
def abs_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true)

In [ ]:
X = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/X.csv')
Xtest = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Xtest.csv')
Y = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Y.csv')

In [ ]:
cols_drop = ['TOLLGATE_ID','INTERSECTION_ID','TRN_STA','TRN_END','TST_STA','XTST_END']
X.drop(cols_drop,axis=1,inplace=True)
Xtest.drop(cols_drop,axis=1,inplace=True)

In [ ]:
## #################### # # Feature selection######################################

In [ ]:
X.head()

In [ ]:
pd.set_option("display.max_columns",100)

In [ ]:
cols = list(X.columns)

In [ ]:
X.columns

In [ ]:
Xtest.columns

In [ ]:
DEL_COLS = ['TIME_MIN','TIME_MAX','TSTW02','TSTW03','TSTW04','TRNW02','TRNW03','TRNW04'
            ,'ROUTE_MIN_TTIME','ROUTE_MAX_TTIME','ROUTE_LAST_TTIME','ROUTE_MEAN_TTIME','FIX']

In [ ]:
for INDEX in DEL_COLS:
    cols.remove(INDEX) 

In [ ]:
len(cols)

In [ ]:
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
# X_F = pd.DataFrame(poly.fit_transform(X[cols]))
# Xtest_F = pd.DataFrame(poly.transform(Xtest[cols]))
# for a in X_F.columns:
#     scaler = StandardScaler()
#     X_F[a] = scaler.fit_transform(X_F[a])
# for a in Xtest_F.columns:
#     scaler = StandardScaler()
#     Xtest_F[a] = scaler.fit_transform(Xtest_F[a])

In [ ]:
X.fillna(method='ffill',inplace=True)

In [ ]:
for a in cols:
    scaler = StandardScaler()
    X[a] = scaler.fit_transform(X[a])
for a in cols:
    scaler = StandardScaler()
    Xtest[a] = scaler.fit_transform(Xtest[a])

In [ ]:
## #################### XGBOOST CV######################################

In [ ]:
import xgboost as xgb
depth = 5
eta = 0.03
mcw = 2
params = {"objective": "reg:linear",
          "booster": "gbtree",
          "eta": eta,
          "max_depth": depth,
          "min_child_weight": mcw,
#               "subsample": 0.9,
#               "colsample_bytree": 0.7,
          "silent": 1
          ,"reg_lambda":1
          }
for ind in range(6):
    dtrain = xgb.DMatrix(X[cols].values,Y.values[:,ind])
    xgb.cv(params,dtrain, num_boost_round=1600,nfold=10,early_stopping_rounds=100,verbose_eval=100)

In [ ]:
## #################### # # ######################################

In [108]:
# 重复划分10次，取平均
# ERROR_LIST=[]
# for x in range(10):
#     ############Split Train DATA and Test Data############################################################
#     from sklearn import cross_validation
#     X_train_m, X_test_m,y_train_m, y_test_m = cross_validation.train_test_split(X,Y, test_size=0.008)
#     ############Split Train DATA and Test Data############################################################

X =  X[cols]
Y = Y
Xtest = Xtest[cols]

error_list = []
Ytrain_all = pd.DataFrame()
XGBR = xgb.XGBRegressor(max_depth = 3,learning_rate=0.1,n_estimators=500,reg_alpha=0,reg_lambda=1)
for ind in range(6):
    time1 = time.time()
    XGBR.fit(X.values,Y.values[:,ind] )
    y_true = Y.values[:,ind]
    y_pred = XGBR.predict(X.values)   
    time2 = time.time()  
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))

    Ytrain =  XGBR.predict(X.values)
    Ytrain_df = pd.DataFrame(Ytrain)
    Ytrain_all = pd.concat((Ytrain_all,Ytrain_df),axis = 1)   
print(np.mean(error_list))

#%%
Ytrain_true = pd.DataFrame(Y)
Y_error = abs_MAPE_error_element(Ytrain_all.values,Ytrain_true.values  )  
Good_ind =  Y_error.sum(axis=1).argsort()[0:np.int(0.90*len(X))]
#%%
XGBR = xgb.XGBRegressor(max_depth = 5,learning_rate=0.03,n_estimators=1600,reg_alpha=1,reg_lambda=0,min_child_weight=2*20)
Ytest_all = pd.DataFrame()
error_list = []
Ytrain_all = pd.DataFrame()
for ind in range(6):
    time1 = time.time()
    XGBR.fit(X.values[Good_ind],Y.values[Good_ind,ind])
    y_true = Y.values[Good_ind,ind]
    y_pred = XGBR.predict(X.values[Good_ind])
    time2 = time.time()    
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))
    Ytest = XGBR.predict(Xtest.values)
    Ytest_df = pd.DataFrame(Ytest)
    Ytest_all = pd.concat((Ytest_all,Ytest_df),axis = 1)    
print(np.mean(error_list))

#     y_test_m.reset_index(drop=True,inplace=True)
#     ERROR_LIST.append(abs_MAPE_error(Ytest_all.values,y_test_m.values))
#     print("train_error:",np.mean(error_list)," vaild_error:",abs_MAPE_error(Ytest_all.values,y_test_m.values))
# print(np.mean(ERROR_LIST))    

0_error:0.16749721002__time:2.568830728530884
1_error:0.169412870864__time:2.5698344707489014
2_error:0.171556546391__time:3.051614999771118
3_error:0.172857111992__time:2.8500797748565674
4_error:0.173910666598__time:2.626986265182495
5_error:0.174852903433__time:2.5171942710876465
0.171681218216
0_error:0.134663311216__time:13.004585266113281
1_error:0.132397864036__time:12.477182388305664
2_error:0.132320501539__time:13.789673566818237
3_error:0.12673540596__time:13.550036907196045
4_error:0.131204817102__time:13.714974164962769
5_error:0.13139579449__time:13.492883205413818
0.131452949057


In [109]:
# Gernerate Final Result

In [110]:
Xtest = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Xtest.csv')
Ytest_all.columns = [0,1,2,3,4,5]
result = pd.concat([Ytest_all,Xtest],axis=1)
result = result.loc[:,[0,1,2,3,4,5,'TOLLGATE_ID','INTERSECTION_ID','TRN_STA','TRN_END']]
result['TRN_END'] = result['TRN_STA'].apply(lambda x :
str((datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours=2))))

In [111]:
result

,0,1,2,3,4,5,TOLLGATE_ID,INTERSECTION_ID,TRN_STA,TRN_END
0,100.047379,81.544846,50.909382,54.535751,65.834267,67.249519,2,A,2016-10-18 06:00:00,2016-10-18 08:00:00
1,176.090439,164.328079,130.809982,125.463135,106.251770,116.174461,3,A,2016-10-18 06:00:00,2016-10-18 08:00:00
2,95.720497,105.576065,90.736832,88.843697,88.595589,89.612236,1,B,2016-10-18 06:00:00,2016-10-18 08:00:00
3,71.049316,69.548119,73.280220,71.776016,58.774078,57.548744,3,B,2016-10-18 06:00:00,2016-10-18 08:00:00
4,194.261810,150.347580,149.883057,159.974655,168.339096,180.987854,1,C,2016-10-18 06:00:00,2016-10-18 08:00:00
5,162.242249,135.349640,136.080627,138.289200,138.483948,129.810455,3,C,2016-10-18 06:00:00,2016-10-18 08:00:00
6,112.043556,72.564186,67.142906,60.619209,63.460438,64.133987,2,A,2016-10-19 06:00:00,2016-10-19 08:00:00
7,155.051865,150.331802,128.577789,124.326218,108.617714,121.646423,3,A,2016-10-19 06:00:00,2016-10-19 08:00:00
8,100.875504,86.658821,90.148140,102.566010,106.048660,87.765114,1,B,2016-10-19 06:00:00,2016-10-19 08:00:00
9,77.008537,77.276154,83.392471,82.182228,78.809525,82.549072,3,B,2016-10-19 06:00:00,2016-10-19 08:00:00


In [112]:
sub_result = pd.DataFrame()
for x in range(result.shape[0]):
    t = result.loc[x,:]
    for y in range(6):
        end_time = str(datetime.datetime.strptime(t['TRN_END'],'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=20*(y+1)))
        start_time = str((datetime.datetime.strptime(t['TRN_END'],'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=20*y)))
        timewindow = '['+start_time+','+end_time+')'
        each_result = pd.DataFrame({'intersection_id':[t['INTERSECTION_ID']],'tollgate_id':[t['TOLLGATE_ID']]
        ,'time_window':[timewindow] ,'avg_travel_time':[t[y]]})  
        sub_result = pd.concat((sub_result,each_result),axis=0)

In [113]:
sub_result

,avg_travel_time,intersection_id,time_window,tollgate_id
0,100.047379,A,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",2
0,81.544846,A,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",2
0,50.909382,A,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",2
0,54.535751,A,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",2
0,65.834267,A,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",2
0,67.249519,A,"[2016-10-18 09:40:00,2016-10-18 10:00:00)",2
0,176.090439,A,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",3
0,164.328079,A,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",3
0,130.809982,A,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",3
0,125.463135,A,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",3


In [114]:
cols = ['intersection_id','tollgate_id','time_window','avg_travel_time']
sub_result = sub_result.ix[:, cols]

In [115]:
sub_result.to_csv('XGBOOST1.csv',index=False,)